# SlopOne预测评分算法

In [1]:
from itertools import product
import pandas as pd
import numpy as np
from collections import defaultdict
from functools import partial

## MovieLens数据读取

In [2]:
data = pd.read_csv('./ratings_small.csv', usecols=['userId', 'movieId', 'rating'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100004 non-null  int64  
 1   movieId  100004 non-null  int64  
 2   rating   100004 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


# SlopOne预测评分算法

In [51]:
class SlopOne():
    def __init__(self):
        pass
        
        
    def fit(self, data, userid, itemid, rating):
        self.data = data
        self.user_col = userid
        self.item_col = itemid
        self.rating_col = rating
        self.all_itemid = data[itemid].unique().tolist()
        self.all_userid = data[userid].unique().tolist()
        
    def predict(self, userid, itemid):
        if self.is_exist(userid, itemid):
            return self.get_predict_score(userid, itemid)
        else:
            pass
        
    def get_predict_score(self, userid, itemid):
        self.score_dict = defaultdict(partial(defaultdict,int))

        for i in self.rating_item:
            for user, groupby_user in self.data.groupby(self.user_col):
                unique_item = groupby_user[self.item_col].unique().tolist()
                if i in unique_item and itemid in unique_item:
                    itemid_rating = groupby_user[groupby_user[self.item_col] == itemid][self.rating_col].values[0]
                    i_rating = groupby_user[groupby_user[self.item_col] == i][self.rating_col].values[0]
                    self.score_dict[(itemid, i)]['sum'] += (itemid_rating - i_rating)
                    self.score_dict[(itemid, i)]['count'] += 1
                else:
                    continue
        score = 0
        for item in self.rating_item:
            if (itemid, item) in self.score_dict:
                score += self.rating_item[item] + self.score_dict[(itemid, item)]['sum'] / self.score_dict[(itemid, item)]['count']
        score = score / len(self.rating_item)
        return round(score, 2)
         
    def get_user_rating_item(self, userid):
        self.user_data = self.data[self.data[self.user_col] == userid]
        return {x:y for x, y in zip(self.user_data[self.item_col].values, self.user_data[self.rating_col].values)}
        
    def is_exist(self, userid, itemid):
        self.rating_item = self.get_user_rating_item(userid)
        
        if userid not in self.all_userid:
            print('用户不存在')
            return False
        
        elif itemid not in self.all_itemid:
            print('商品不存在')
            return False
        
        elif itemid in self.rating_item :
            print('该用户已经对此商品打过分')
            return False

        else:
            return True

In [52]:
rec = SlopOne()
rec.fit(data, userid='userId', itemid='movieId', rating='rating')
rec.predict(1,35)

1.68